In [16]:
from langchain_pinecone import PineconeVectorStore
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
# from sentence_transformers import SentenceTransformer
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_pinecone import PineconeVectorStore
from langchain.chains.question_answering import load_qa_chain
from langchain import PromptTemplate
from uuid import uuid4
from langchain_core.documents import Document
import getpass
import os
import google.generativeai as genai
from langchain.document_loaders import PyPDFLoader

### Importing the documents needs to be loaded in Vector Database


In [2]:
base_dir = os.getcwd()
file_name = "Reliance.pdf"
file_path = os.path.join(base_dir,file_name )

def create_chunks(doc_to_chunk):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=100,
        length_function=len
    )
    return text_splitter.split_documents(doc_to_chunk)

def load_pdf(path):
    loader = PyPDFLoader(path)
    return loader.load()

def load_chunk_file(path):
    doc = load_pdf(path)
    return create_chunks(doc)

chunks = load_chunk_file(file_path)

### Loading the document into Vector Database

In [3]:
# Import the necessary modules
from langchain_pinecone import PineconeVectorStore
from langchain_core.documents import Document
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv
import os
from pinecone import Pinecone, ServerlessSpec
load_dotenv()

# Initialize the Pinecone vector store
index_name = "test-index"
pc = Pinecone(
        api_key=os.environ["PINECONE_API_KEY"]) 
if index_name not in pc.list_indexes().names():
        pc.create_index(
            name=index_name,
            dimension=768,
            metric='cosine',  
            spec=ServerlessSpec(
                cloud='aws',  # Specify your preferred cloud provider
                region='us-east-1'  # Specify your preferred region
            )
        )

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=os.environ["GOOGLE_API_KEY"])
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)
uuids = [str(uuid4()) for _ in range(len(chunks))]
vectorstore.add_documents(documents=chunks, ids=uuids)




['0c2ac5d0-8ee3-4b7e-8846-2918154f1eb8',
 'aa5fc10a-7127-4f21-889f-28f4416f476b',
 '1642ce19-52a7-47ba-b9f5-571427234765',
 'ab61277d-6c32-4c2b-9f16-462e1d91920b',
 'df66c977-49f0-4f0c-b15d-399bae4cd22f',
 '2a7a3525-6032-4c9d-8841-8ae7ebf56850',
 'b43a8564-a85b-4119-b58d-575e45a42165',
 '30ab4aba-7d2c-47b4-834c-59bc92f9ed74',
 '623f618b-a7f2-4095-b115-8b410ea9eb9e',
 'c2ea7159-4de7-448e-aa69-1e9accff1850',
 '5869be42-c13a-4370-bc3b-2007d09cb7bc',
 'e00749b7-b2a5-42f1-a16e-d6ed85849c91',
 '3673b32b-14b0-479d-8791-12f22bd90ff8',
 '919227ed-2b17-4b9c-a7d8-1de38c14d626',
 'e9af81a2-0753-4788-993d-b59cc2cada2c',
 '872a0391-ad12-4eb5-baa5-fe6fe8ee659b',
 '744cf651-e62d-4739-9531-a0dc8e4f1f07',
 '645c60da-b40a-453a-b18f-4c06d314e221',
 'b4f052a5-475c-43ab-8252-b28793ccd5c4',
 'c40aea8d-d4a2-4af4-9054-97613d23e4a0',
 'ea57becd-f898-47bb-9ca5-9fe15cb20454',
 'd6b72ea5-eb18-47c0-8662-d4d211dfd10b',
 '83cea095-1704-46b1-85bd-79888fdf7d86',
 '189bdbb1-a31a-4e66-b5e8-59b1b4655398',
 'a34b2aed-81a9-

### Create questions for the performance evaluation of RAG

In [8]:
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.gemini import GeminiEmbeddings


ModuleNotFoundError: No module named 'llama_index.embeddings.gemini'

In [28]:
from dotenv import load_dotenv
load_dotenv()
resp = Gemini(Model = "gemini-1.5-flash").complete("Write a poem about a magic backpack")
print(resp)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\agshi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\agshi\AppData\Local\Temp\ipykernel_38684\3905429830.py", line 3, in <module>
    resp = Gemini(Model = "gemini-1.5-flash").complete("Write a poem about a magic backpack")
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\agshi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\llama_index\llms\gemini\base.py", line 147, in __init__
    model_meta = genai.get_model(model)
                 ^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\agshi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\google\generativeai\models.py", line 55, i

In [20]:
GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"]

In [4]:
questions = [
    "What is the company's registered office address?",
    "What is the phone number of the company?",
    "When was the company established?",
    "Who is the regulatory authority for the company?",
    "What is the website of the company?",
    "What is the CIN (Corporate Identification Number) of the company?",
    "Who is the CEO of the company?",
    "What is the company's market capitalization?",
    "What are the major products or services offered by the company?",
    "What is the company's financial performance in the last fiscal year?"
]

In [13]:
from trulens_eval.feedback.provider.langchain import Langchain
from trulens_eval.feedback.provider.hugs import Huggingface
from trulens_eval import Feedback
from trulens_eval import TruLlama
context_selection = TruLlama.select_source_nodes().node.text
provider = Huggingface()
import numpy as np
huggingface_provider = Huggingface()
# Define a groundedness feedback function
f_groundedness = (
    Feedback(huggingface_provider.groundedness_measure_with_nli,
             name="Context Relevance")
    .on_input()
    .on(context_selection)
    .aggregate(np.mean)
)
# Question/answer relevance between overall question and answer.
f_answer_relevance = Feedback(
    huggingface_provider.context_relevance,
    name="Answer Relevance"
).on_input_output()

# Context relevance between question and each context chunk.
f_context_relevance = (
    Feedback(huggingface_provider.context_relevance,
             name="Context Relevance")
    .on_input()
    .on(context_selection)
    .aggregate(np.mean)
)

✅ In Context Relevance, input source will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input context will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.app.query.rets.source_nodes[:].node.text .


In [12]:
from trulens_eval.feedback.provider.hugs import Huggingface
huggingface_provider = Huggingface()

In [14]:
def generate_answer(query, vector_store, template):
    # Use LangChain's RetrievalQA to query the vector store and generate the answer
    qa_chain = RetrievalQA.from_chain_type(
        llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash"),
        chain_type="stuff",
        retriever=vector_store.as_retriever(search_type="similarity")
    )

    # Format the prompt with the retrieved context and the query
    prompt = template.format(context="{context}", question=query)
    
    # Generate the answer by running the chain with the combined prompt
    answer = qa_chain.run(query=query)  # Pass 'input' as keyword argument with the formatted prompt
    return answer

In [ ]:
generate_answer(
    "How do you create your AI portfolio?")

In [15]:
from trulens_eval import TruLlama
from trulens_eval import FeedbackMode
tru_recorder = TruLlama(
    generate_answer,
    app_id="App_1",
    feedbacks=[
        f_groundedness,
        f_answer_relevance,
        f_context_relevance
    ]
)

ValidationError: 2 validation errors for TruLlama
app.is-instance[BaseQueryEngine]
  Input should be an instance of BaseQueryEngine [type=is_instance_of, input_value=<function generate_answer at 0x0000019D4B98EFC0>, input_type=function]
    For further information visit https://errors.pydantic.dev/2.8/v/is_instance_of
app.is-instance[BaseChatEngine]
  Input should be an instance of BaseChatEngine [type=is_instance_of, input_value=<function generate_answer at 0x0000019D4B98EFC0>, input_type=function]
    For further information visit https://errors.pydantic.dev/2.8/v/is_instance_of